## Task 1 : Import libraries and connect to the database

In [9]:
import os
import json
import oracledb
import pandas as pd
import oci
import numpy as np
import re
from dotenv import load_dotenv

load_dotenv()

username = os.getenv("USERNAME")
password = os.getenv("DBPASSWORD")
dsn = os.getenv("DBCONNECTION")

try:
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")

cursor = connection.cursor()

Connection successful!


## Task 2: Create the tables and load data

In [12]:
DROP_OBJECTS_SQL = [
    "DROP VIEW IF EXISTS CLIENT_DV",
    "DROP PROPERTY GRAPH IF EXISTS LOANS_GRAPH_VW",
    "DROP TABLE IF EXISTS CLIENTS_TO_LOAN_RECOMMENDATIONS CASCADE CONSTRAINTS PURGE",
    "DROP TABLE IF EXISTS CLIENTS_TO_LOAN CASCADE CONSTRAINTS PURGE",
    "DROP TABLE IF EXISTS CLIENT_DEBT CASCADE CONSTRAINTS PURGE",
    "DROP TABLE IF EXISTS LOAN_CHUNK CASCADE CONSTRAINTS PURGE",
    "DROP TABLE IF EXISTS LOAN_APPLICATIONS CASCADE CONSTRAINTS PURGE",
    "DROP TABLE IF EXISTS MOCK_LOAN_DATA CASCADE CONSTRAINTS PURGE",
    "DROP TABLE IF EXISTS CLIENTS CASCADE CONSTRAINTS PURGE",
    "DROP TABLE IF EXISTS FUNDING_PROVIDER CASCADE CONSTRAINTS PURGE",
    "DROP TABLE IF EXISTS FUNDING_PROVIDER_TERMS CASCADE CONSTRAINTS PURGE",
    "DROP TABLE IF EXISTS LENDER_TERMS CASCADE CONSTRAINTS PURGE",
    "DROP TABLE IF EXISTS AFFORDABLE_HOUSING_ZONE CASCADE CONSTRAINTS PURGE",
    "DROP TABLE IF EXISTS FLOODZONE CASCADE CONSTRAINTS PURGE",
]

CREATE_CLIENTS = """
CREATE TABLE IF NOT EXISTS CLIENTS (
    CUSTOMER_ID VARCHAR2(4000) PRIMARY KEY,
    FIRST_NAME  VARCHAR2(4000),
    LAST_NAME   VARCHAR2(4000),
    CITY        VARCHAR2(4000),
    STATE       VARCHAR2(4000),
    ZIP_CODE    NUMBER,
    AGE         NUMBER,
    INCOME      NUMBER,
    VETERAN     VARCHAR2(4000)
)
"""

CREATE_LOAN_APPLICATIONS = """
CREATE TABLE IF NOT EXISTS LOAN_APPLICATIONS (
    CUSTOMER_ID VARCHAR2(4000),
    APPLICATION_ID NUMBER GENERATED BY DEFAULT AS IDENTITY (START WITH 1000 INCREMENT BY 1000) PRIMARY KEY,
    REQUESTED_LOAN_AMOUNT NUMBER,
    CREDIT_SCORE NUMBER,
    ZIPCODE NUMBER,
    LOAN_PURPOSE VARCHAR2(4000),
    LOAN_STATUS VARCHAR2(4000),
    STUDENT_STATUS VARCHAR2(4000),
    EDUCATION_LEVEL VARCHAR2(4000),
    FINAL_DECISION VARCHAR2(32767),
    RECOMMENDATIONS VARCHAR2(32767),
    TOTAL_DEBT NUMBER,
    CREDIT_RANK NUMBER DEFAULT NULL,
    CONSTRAINT fk_loan_app_clients FOREIGN KEY (CUSTOMER_ID) REFERENCES CLIENTS(CUSTOMER_ID)
)
"""

CREATE_MOCK_LOAN_DATA = """
CREATE TABLE IF NOT EXISTS MOCK_LOAN_DATA (
    LOAN_ID NUMBER PRIMARY KEY,
    LOAN_PROVIDER_NAME VARCHAR2(4000),
    LOAN_TYPE VARCHAR2(4000),
    INTEREST_RATE NUMBER,
    ORIGINATION_FEE NUMBER,
    TIME_TO_CLOSE NUMBER,
    CREDIT_SCORE NUMBER,
    DEBT_TO_INCOME_RATIO NUMBER,
    INCOME NUMBER,
    DOWN_PAYMENT_PERCENT NUMBER,
    IS_FIRST_TIME_HOME_BUYER VARCHAR2(10) NOT NULL,
    OFFER_BEGIN_DATE DATE DEFAULT SYSDATE,
    OFFER_END_DATE DATE DEFAULT SYSDATE + 30
)
"""

CREATE_CLIENT_DEBT = """
CREATE TABLE IF NOT EXISTS CLIENT_DEBT (
    ID NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    CUSTOMER_ID VARCHAR2(100),
    APPLICATION_ID NUMBER,
    DEBT_TYPE VARCHAR2(100),
    DEBT_AMOUNT NUMBER
)
"""

CREATE_CLIENTS_TO_LOAN = """
CREATE TABLE IF NOT EXISTS CLIENTS_TO_LOAN (
    ID NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    CUSTOMER_ID VARCHAR2(4000),
    LOAN_ID NUMBER,
    LOAN_APPLICATION_ID NUMBER,
    CONSTRAINT fk_ctl_clients FOREIGN KEY (CUSTOMER_ID) REFERENCES CLIENTS(CUSTOMER_ID),
    CONSTRAINT fk_ctl_loans FOREIGN KEY (LOAN_ID) REFERENCES MOCK_LOAN_DATA(LOAN_ID),
    CONSTRAINT fk_ctl_loan_applications FOREIGN KEY (LOAN_APPLICATION_ID) REFERENCES LOAN_APPLICATIONS(APPLICATION_ID)
)
"""

CREATE_CLIENTS_TO_LOAN_RECS = """
CREATE TABLE IF NOT EXISTS CLIENTS_TO_LOAN_RECOMMENDATIONS (
    ID NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    CUSTOMER_ID VARCHAR2(4000),
    LOAN_ID NUMBER,
    LOAN_APPLICATION_ID NUMBER,
    ACTION_NEEDED VARCHAR2(4000),
    CONSTRAINT fk_cltr_clients FOREIGN KEY (CUSTOMER_ID) REFERENCES CLIENTS(CUSTOMER_ID),
    CONSTRAINT fk_cltr_loans FOREIGN KEY (LOAN_ID) REFERENCES MOCK_LOAN_DATA(LOAN_ID),
    CONSTRAINT fk_cltr_loan_applications FOREIGN KEY (LOAN_APPLICATION_ID) REFERENCES LOAN_APPLICATIONS(APPLICATION_ID)
)
"""

CREATE_LOAN_CHUNK = """
CREATE TABLE IF NOT EXISTS LOAN_CHUNK (
    ID NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    CUSTOMER_ID VARCHAR2(100),
    CHUNK_ID NUMBER,
    CHUNK_TEXT CLOB,
    CHUNK_VECTOR VECTOR(384,*,DENSE)
)
"""

CREATE_FUNDING_PROVIDER_TERMS = """
CREATE TABLE IF NOT EXISTS FUNDING_PROVIDER_TERMS (
    LOAN_PROVIDER_ID NUMBER,
    TERMS_ID NUMBER PRIMARY KEY,
    INTEREST_RATE NUMBER,
    LOAN_DESCRIPTION VARCHAR2(1000),
    TIME_TO_CLOSE NUMBER,
    LOAN_COSTS NUMBER,
    OFFER_BEGIN_DATE DATE,
    OFFER_END_DATE DATE
)
"""

CREATE_LENDER_TERMS = """
CREATE TABLE IF NOT EXISTS LENDER_TERMS (
    LENDER_ID NUMBER,
    TERMS_ID NUMBER PRIMARY KEY,
    LOAN_DESCRIPTION VARCHAR2(1000),
    INTEREST_RATE_MARKUP NUMBER,
    ORIGINATION_FEE NUMBER,
    LENDER_TIME_TO_CLOSE NUMBER,
    CREDIT_SCORE NUMBER,
    DEBT_TO_INCOME_RATIO NUMBER,
    INCOME NUMBER,
    DOWN_PAYMENT_PERCENT NUMBER,
    OFFER_BEGIN_DATE DATE,
    OFFER_END_DATE DATE,
    CONSTRAINT fk_lt_mock_loan_data FOREIGN KEY (LENDER_ID) REFERENCES MOCK_LOAN_DATA(LOAN_ID)
)
"""

CREATE_AFFORDABLE_HOUSING_ZONE = """
CREATE TABLE IF NOT EXISTS AFFORDABLE_HOUSING_ZONE (
    LAND_TRACT_ID NUMBER PRIMARY KEY,
    ZIPCODE NUMBER
)
"""

CREATE_FLOODZONE = """
CREATE TABLE IF NOT EXISTS FLOODZONE (
    GEOMETRY MDSYS.SDO_GEOMETRY,
    FID NUMBER,
    DESCR VARCHAR2(32767)
)
"""

# Create JSON Duality View

CREATE_CLIENTS_DV = """
CREATE OR REPLACE JSON RELATIONAL DUALITY VIEW clients_dv AS
SELECT JSON {
    '_id': c.customer_id,
    'firstName': c.first_name,
    'lastName': c.last_name,
    'city': c.city,
    'state': c.state,
    'zipCode': c.zip_code,
    'age': c.age,
    'income': c.income,
    'veteran': c.veteran,
    'clientDebt': [
        SELECT JSON{
            'id': cd.id,
            'debtType': cd.debt_type,
            'debtAmount': cd.debt_amount
        }
        FROM client_debt cd WITH INSERT UPDATE DELETE
        WHERE cd.customer_id = c.customer_id
    ],
    'loanApplications': [
        SELECT JSON {
            'applicationId': la.application_id,
            'requestedLoanAmount': la.requested_loan_amount,
            'creditScore': la.credit_score,
            'zipcode': la.zipcode,
            'loanPurpose': la.loan_purpose,
            'loanStatus': la.loan_status,
            'studentStatus': la.student_status,
            'educationLevel': la.education_level,
            'finalDecision': la.final_decision,
            'recommendations': la.recommendations,
            'totalDebt' : la.total_debt,
            'creditRank': la.credit_rank
        }
        FROM loan_applications la WITH INSERT UPDATE DELETE
        WHERE la.customer_id = c.customer_id
    ]
}
FROM clients c WITH INSERT UPDATE DELETE
"""

# Load ONNX Model (DEMO_MODEL)

ONNX_MODEL_SQL = """
DECLARE
    model_count INT;
BEGIN
    SELECT COUNT(*) INTO model_count
    FROM user_mining_models
    WHERE model_name = 'DEMO_MODEL';
    IF (model_count < 1) THEN
        EXECUTE IMMEDIATE 'CREATE OR REPLACE DIRECTORY DEMO_DIR AS ''demodir''';
        DBMS_CLOUD.GET_OBJECT(
            object_uri => 'https://objectstorage.us-ashburn-1.oraclecloud.com/p/mFBJq8UCjdar89xJpTQVOy_tONdxyrQI-B8UT0OS-nllmg8xyVAIIOTbGyIVJ1iJ/n/c4u04/b/llm/o/all_MiniLM_L12_v2.onnx',
            directory_name => 'DEMO_DIR',
            file_name => 'all_minilm_l12_v2.onnx'
        );
        DBMS_DATA_MINING.DROP_MODEL(model_name => 'DEMO_MODEL', force => TRUE);
        DBMS_VECTOR.LOAD_ONNX_MODEL('DEMO_DIR', 'all_minilm_l12_v2.onnx', 'DEMO_MODEL');
    END IF;
END;
"""


# Sample Data 

AFFORDABLE_HOUSING_ZONE_DATA = [(1, 48202)]

CLIENTS_DATA = [
    ('CUST_1000','James','Smith','New York City','NY',10033,52,130000,'Yes'),
    ('CUST_2000','James','Woods','East Gina','AR',58967,27,5000,'Yes'),
    ('CUST_3000','Evan','Burton','Detroit','MI',48202,24,65000,'No'),
    ('CUST_4000','Alex','Anderson','Austin','TX',78744,26,25000,'No'),
    ('CUST_5000','Richard','Thomas','Chicago','IL',60127,27,80000,'Yes'),
    ('CUST_6000','Karen','Smith','San Jose','CA',94560,42,115000,'No'),
    ('CUST_7000','Omar','Santos','Orlando','FL',32802,34,25000,'No'),
    ('CUST_8001','Billy','Bob','Chicago','IL',60127,43,90000,'Yes'),
    ('CUST_8002','Johnny','Appleseed','San Jose','CA',94560,41,120000,'No'),
    ('CUST_8003','Susie','Bee','Orlando','FL',32802,30,85000,'No'),
    ('CUST_8004','Money','Mike','Chicago','IL',60127,35,120000,'Yes'),
]

LOAN_APPLICATIONS_DATA = [
    ('CUST_1000',1000,200000,780,10033,'Mortgage','Pending Review','No','PhD',None,None,None,0),
    ('CUST_2000',1001,100000,520,58967,'Mortgage','Pending Review','No','High School',None,None,None,0),
    ('CUST_3000',1002,150000,675,48202,'Mortgage','Pending Review','No','Masters',None,None,None,0),
    ('CUST_4000',1003,300000,700,78744,'Mortgage','Pending Review','No','Bachelor',None,None,None,0),
    ('CUST_5000',1004,95000,680,60127,'Mortgage','Approved','No','PhD',None,None,None,0),
    ('CUST_6000',1005,200000,700,94560,'Mortgage','Denied','Yes','Bachelor',None,None,None,0),
    ('CUST_7000',1006,200000,730,32802,'Mortgage','In Progress','No','Masters',None,None,None,0),
    ('CUST_8001',8001,200000,750,10033,'Mortgage','Pending Review','No','PhD',None,None,None,0),
    ('CUST_8002',8002,200000,730,48202,'Mortgage','Pending Review','Yes','Bachelor',None,None,None,0),
    ('CUST_8003',8003,200000,780,10033,'Mortgage','Pending Review','No','Masters',None,None,None,0),
    ('CUST_8004',8004,200000,780,48202,'Mortgage','In Progress','No','Masters',None,None,None,0),
]


MOCK_LOAN_DATA = [
    (1,'provider1','Bridge loan for transitional homebuyers',4.75,1.061810178,38,649,27.15,150377,12.43,'NO'),
    (2,'provider2','Second mortgage loan',3.31,1.92607146,25,776,28.81,154763,3.76,'NO'),
    (3,'provider3','Low interest rate for high credit',3.36,1.597990913,22,712,49.88,90067,12.25,'NO'),
    (4,'provider4','Balloon payment loan',5.69,1.397987726,50,668,42.94,194469,13.53,'NO'),
    (5,'provider5','First time home owner loan (3% down, good credit, first home)',3.59,0.734027961,52,730,28.93,84122,9.86,'NO'),
    (6,'provider6','Prepaid points loan (3% if buying 2% points)',6.07,0.733991781,54,746,46.42,67695,10.10,'NO'),
    (7,'provider7','First Time Home Owner loan (low down, low credit)',3.33,0.587125418,34,727,42.06,143050,0.92,'NO'),
    (8,'provider1','Cash-out refinance loan',4.17,1.799264219,49,798,46.13,70436,14.40,'NO'),
    (9,'provider2','Balloon payment loan',5.94,1.401672518,39,631,48.90,109708,17.12,'NO'),
    (10,'provider3','First time home owner loan (3% down, good credit, first home)',3.05,1.562108867,49,793,38.51,59700,17.06,'NO'),
    (11,'provider4','HELOC (Home Equity Line of Credit)',3.47,0.530876741,39,749,32.93,101423,18.59,'NO'),
    (12,'provider5','Equity loan',3.92,1.954864778,43,626,26.43,61053,12.52,'NO'),
    (13,'provider6','Standard low interest with standard money down',4.63,1.748663961,32,602,41.59,95781,2.08,'NO'),
    (14,'provider7','Refinance loan',5.48,0.818508666,60,670,22.73,86702,0.86,'NO'),
    (15,'provider1','Stated income loan (for self-employed)',4.19,0.772737451,32,613,22.15,134070,4,'NO'),
    (16,'provider2','Reverse mortgage (for seniors)',4.95,0.775106765,16,789,48.43,87550,12.01,'NO'),
    (17,'provider3','Low down payment for good credit',5.18,0.956363364,49,674,44.9,111661,12.83,'NO'),
    (18,'provider4','Personal loan for home improvement',5.33,1.287134647,30,611,36.45,87071,10.71,'NO'),
    (19,'provider5','Adjustable rate mortgage (ARM)',5.27,1.147917528,55,674,34.67,192887,12.15,'NO'),
    (20,'provider6','Construction loan for new builds',6.17,0.93684371,50,731,28.6,171741,15.36,'NO'),
    (21,'provider7','Balloon payment loan',6.41,1.417779342,47,658,46.6,140958,17.46,'NO'),
    (22,'provider1','Jumbo loan for high-Military Veteranlue properties',5.07,0.709240791,18,767,23.51,134693,1.4,'NO'),
    (23,'provider2','First time home owner loan (3% down, good credit, first home)',4.26,0.938216973,47,669,30.78,65253,4.35,'YES'),
    (24,'provider3','Construction loan for new builds',3.64,1.049542765,28,773,43.17,86011,6.97,'NO'),
    (25,'provider4','Interest-only loan',5.25,1.184104976,35,676,27.15,91273,5.25,'NO'),
    (26,'provider5','Military Veteran loan (0% down, military only)',3.47,1.677763942,34,695,41.18,160054,10.57,'NO'),
    (27,'provider6','First time home owner loan (3% down, good credit, first home)',4.03,0.799510673,22,784,39.03,172398,8.7,'YES'),
    (28,'provider7','Affordable housing zone loan (1% interest, max income $100k)',1.0,1.271351658,21,717,27.73,100000,8.02,'NO'),
    (29,'provider1','Construction loan for new builds',4.88,1.388621853,17,674,41.65,153775,5.17,'NO'),
    (30,'provider2','Stated income loan (for self-employed)',4.81,0.569675619,31,787,35.58,76574,2.95,'NO'),
    (31,'provider3','Debt consolidation loan',3.92,1.411317278,47,655,36.31,134170,13.89,'NO'),
    (32,'provider4','Reverse mortgage (for seniors)',4.6,0.755786186,26,738,21.66,180259,0.62,'NO'),
    (33,'provider5','Low interest rate for high credit',5.61,0.597577389,36,611,40.00,149496,19.77,'NO'),
    (34,'provider6','Equity loan',5.94,1.923328306,36,647,47.06,162647,15.81,'NO'),
    (35,'provider7','Down payment assistance loan',3.50,1.94844805,60,663,24.92,187270,4.43,'NO'),
    (36,'provider1','Refinance loan',6.09,1.712596022,44,788,40.14,124887,1.73,'NO'),
    (37,'provider2','Interest-only loan',6.45,0.956920654,52,680,20.03,129807,15.33,'NO'),
    (38,'provider3','Personal loan for home improvement',3.93,0.646508171,52,635,48.27,74941,1.74,'NO'),
    (39,'provider4','No closing cost loan',4.12,1.52634954,59,713,23.74,123424,0.47,'NO'),
    (40,'provider5','Second mortgage loan',5.91,1.160228741,22,682,24.93,57743,9.31,'NO'),
    (41,'provider6','Interest-only loan',3.24,0.683057352,41,621,23.39,147609,19.36,'NO'),
    (42,'provider7','Medical professional loan (low down, high income potential)',3.94,1.242765365,41,606,33.65,92858,12.38,'NO'),
    (43,'provider1','Stated income loan (for self-employed)',5.23,0.551582782,48,689,42.69,160945,18.65,'NO'),
    (44,'provider2','USDA loan (rural development, low income)',3.49,1.863980603,35,628,49.42,176267,10.14,'NO'),
    (45,'provider3','HELOC (Home Equity Line of Credit)',4.03,0.888169972,44,685,36.07,136753,17.82,'NO'),
    (46,'provider4','USDA loan (rural development, low income)',6.45,1.493783427,47,775,24.93,145916,3.22,'NO'),
    (47,'provider5','Bridge loan for transitional homebuyers',4.11,0.967566614,42,664,45.73,174745,10.23,'NO'),
    (48,'provider6','First time home owner loan (3% down, good credit, first home)',6.38,1.280102032,47,711,33.84,85158,12.90,'YES'),
    (49,'provider7','Affordable housing zone loan (1% interest, max income $100k)',1.0,1.320065419,19,631,35.61,100000,15.35,'NO'),
    (50,'provider1','Military Veteran loan (0% down, military only)',4.75,0.777281683,33,634,27.38,56291,14.27,'NO'),
    (51,'provider2','Second mortgage loan',4.32,1.954376942,18,638,29.34,108990,17.41,'NO'),
    (52,'provider3','Medical professional loan (low down, high income potential)',3.03,1.662699235,49,712,31.89,180376,8.37,'NO'),
    (53,'provider4','Green loan (for energy-efficient homes)',5.00,1.909248412,31,710,41.49,147038,3.05,'NO'),
    (54,'provider5','HELOC (Home Equity Line of Credit)',5.74,1.842241026,58,703,48.08,187286,13.18,'NO'),
    (55,'provider6','RenoMilitary Veterantion loan',5.82,1.396849968,42,739,46.17,88439,2.76,'NO'),
    (56,'provider7','Bridge loan for transitional homebuyers',5.61,1.882811353,44,619,33.11,194147,4.22,'NO'),
    (57,'provider1','Equity loan',5.52,0.632738753,43,653,43.59,108215,2.57,'NO'),
    (58,'provider2','First Time Home Owner loan (low down payment, good for low credit)',5.11,0.793974294,60,679,45.91,135967,3.89,'NO'),
    (59,'provider3','RenoMilitary Veterantion loan',5.98,0.567840933,20,733,46.00,122926,17.27,'NO'),
    (60,'provider4','Interest-only loan',3.14,0.987995496,49,604,24.57,103122,10.37,'NO'),
    (61,'provider5','Equity loan',3.13,1.083015935,55,702,44.40,93247,6.69,'NO'),
    (62,'provider6','Affordable housing zone loan (1% interest, max income $100k)',1.0,0.907023548,51,674,25.82,100000,3.83,'NO'),
    (63,'provider7','Low down payment for good credit',6.15,1.743106264,38,718,32.75,102928,15.88,'NO'),
    (64,'provider1','HELOC (Home Equity Line of Credit)',3.19,1.03512999,43,600,24.93,126203,1.79,'NO'),
    (65,'provider2','Cash-out refinance loan',4.87,0.921401765,60,653,23.02,72697,11.77,'NO'),
    (66,'provider3','Low down payment for good credit',4.60,1.314044125,45,793,22.55,75763,5.94,'NO'),
    (67,'provider4','Low down payment for good credit',5.70,0.711386337,49,636,22.46,142301,18.32,'NO'),
    (68,'provider5','Standard low interest with standard money down',4.60,1.703295471,47,767,23.31,151345,9.27,'NO'),
    (69,'provider6','Balloon payment loan',3.54,0.611825966,35,785,44.26,66442,18.45,'NO'),
    (70,'provider7','Reverse mortgage (for seniors)',4.64,1.980330405,46,716,39.07,131353,19.51,'NO'),
    (71,'provider1','Low down payment for good credit',5.18,1.658367154,37,636,25.11,66991,20.00,'NO'),
    (72,'provider2','Personal loan for home improvement',5.03,0.798073522,47,800,23.64,55497,15.55,'NO'),
    (73,'provider3','Reverse mortgage (for seniors)',3.65,0.508283176,17,720,38.09,178944,18.79,'NO'),
    (74,'provider4','Affordable housing zone loan (1% interest, max income $100k)',1.0,1.723192143,32,736,49.31,100000,18.18,'NO'),
    (75,'provider5','Fixed rate mortgage',4.35,1.560286016,39,687,23.25,161364,14.51,'NO'),
    (76,'provider6','First time home owner loan (3% down, good credit, first home)',4.64,1.593510752,56,707,42.99,51829,19.27,'YES'),
    (77,'provider7','Affordable housing zone loan (1% interest, max income $100k)',1.0,1.65690552,45,788,46.95,100000,8.64,'NO'),
    (78,'provider1','Jumbo loan for high-Military Veteranlue properties',3.99,0.611066978,17,712,37.25,87710,9.75,'NO'),
    (79,'provider2','Equity loan',6.29,1.037698593,54,695,27.1,80990,16.62,'NO'),
    (80,'provider3','Renovation loan',6.21,0.673803589,60,787,32.4,151567,6.42,'NO'),
    (81,'provider4','Fixed rate mortgage',3.2,1.794655139,38,620,43.49,133977,8.3,'NO'),
    (82,'provider5','Fixed rate mortgage',5.64,1.43494719,46,655,28.28,139696,0.02,'NO'),
    (83,'provider6','Medical professional loan (low down, high income potential)',5.86,0.996347037,36,606,27.26,86374,19.63,'NO'),
    (84,'provider7','USDA loan (rural development, low income)',4.81,0.595337525,37,663,23.68,50000,18.7,'NO'),
    (85,'provider1','Affordable housing zone loan (1% interest, max income $100k)',1.0,0.966473483,16,685,41.01,100000,4.57,'NO'),
    (86,'provider2','Low down payment for good credit',4.13,0.987774983,41,792,36.84,68452,12.23,'NO'),
    (87,'provider3','Low interest rate for high credit',4.99,1.594409268,56,625,27.21,158436,9.13,'NO'),
    (88,'provider4','HELOC (Home Equity Line of Credit)',5.77,1.456336207,16,755,32.29,189755,1.01,'NO'),
    (89,'provider5','Adjustable rate mortgage (ARM)',5.72,1.830819114,40,668,32.13,131312,13.62,'NO'),
    (90,'provider6','Affordable housing zone loan (1% interest, max income $100k)',1.0,1.208322388,31,705,24.03,100000,18.13,'NO'),
    (91,'provider7','Reverse mortgage (for seniors)',3.03,0.679391369,54,680,29.3,52503,10.03,'NO'),
    (92,'provider1','No closing cost loan',5.4,1.569867181,47,635,23.75,182156,9.13,'NO'),
    (93,'provider2','Adjustable rate mortgage (ARM)',5.82,1.641177573,23,731,36.57,62875,3.66,'NO'),
    (94,'provider3','Standard low interest with standard money down',6.49,1.341915796,57,703,36.68,64630,11.42,'NO'),
    (95,'provider4','Jumbo loan for high-value properties',6.46,1.65645077,53,640,20.97,130978,9.08,'NO'),
    (96,'provider5','Stated income loan (for self-employed)',3.6,1.240693395,43,770,36.3,151653,5.21,'NO'),
    (97,'provider6','Affordable housing zone loan (1% interest, max income $100k)',1.0,1.284099244,56,633,25.81,100000,2.7,'NO'),
    (98,'provider7','Interest-only loan',4.4,1.141311528,40,678,20.02,138999,2.13,'NO'),
    (99,'provider1','Standard low interest with standard money down',5.24,0.53812869,49,749,24.09,111456,13.07,'NO'),
    (100,'provider2','First time home owner loan (3% down, good credit, first home)',4.23,0.66183714,39,695,23.32,181156,2.76,'YES')
]

CLIENT_DEBT_DATA = [
    ('CUST_1000',1000,'First Time Home Owner',90000),
    ('CUST_2000',1001,'Home',8843),
    ('CUST_2000',1001,'CREDIT_CARD',31560),
    ('CUST_2000',1001,'Business',49253),
    ('CUST_3000',1002,'Personal',5000),
    ('CUST_3000',1002,'Business',22000),
    ('CUST_4000',1003,'Personal',42000),
    ('CUST_5000',1004,'Military Veteran',51712),
    ('CUST_5000',1004,'Conventional',11343),
    ('CUST_5000',1004,'Auto',24147),
    ('CUST_5000',1004,'First Time Home Owner',4802),
    ('CUST_5000',1004,'Personal',5908),
    ('CUST_5000',1004,'Auto',18327),
    ('CUST_5000',1004,'Business',58944),
    ('CUST_5000',1004,'Personal',64862),
    ('CUST_5000',1004,'Conventional',78594),
    ('CUST_6000',1005,'Personal',37761),
    ('CUST_6000',1005,'First Time Home Owner',84400),
    ('CUST_6000',1005,'Military Veteran',41642),
    ('CUST_6000',1005,'Auto',95561),
    ('CUST_6000',1005,'Personal',25820),
    ('CUST_6000',1005,'Conventional',6018),
    ('CUST_6000',1005,'Personal',25738),
    ('CUST_6000',1005,'Personal',75140),
    ('CUST_6000',1005,'Personal',65379),
    ('CUST_7000',1006,'Auto',20000),
    ('CUST_7000',1006,'Auto',10000),
    ('CUST_8001',8001,'Personal',5000),
    ('CUST_8001',8001,'Credit_Card',4000),
    ('CUST_8001',8001,'Auto',30000),
    ('CUST_8002',8002,'Personal',5000),
    ('CUST_8002',8002,'Credit_Card',4000),
    ('CUST_8002',8002,'Auto',30000),
    ('CUST_8003',8003,'Personal',5000),
    ('CUST_8003',8003,'Credit_Card',4000),
    ('CUST_8003',8003,'Auto',30000),
    ('CUST_8004',8004,'Personal',5000),
    ('CUST_8004',8004,'Credit_Card',4000),
    ('CUST_8004',8004,'Auto',30000),
]

CLIENTS_TO_LOAN_RECS_DATA = [
    ('CUST_1000', 64, 'Review Documents'),
    ('CUST_2000', 76, 'Verify Credit'),
    ('CUST_3000', 81, 'Contact Customer'),
    ('CUST_4000', 41, 'Verify Credit'),
    ('CUST_5000', 66, 'Verify Credit'),
    ('CUST_6000', 85, 'Verify Credit'),
    ('CUST_7000', 32, 'Contact Customer'),
    ('CUST_8001', 18, 'None'),
    ('CUST_8002', 97, 'None'),
    ('CUST_8003', 31, 'Review Documents'),
]


# Helpers
def safe_execute(sql: str, params=None):
    try:
        cursor.execute(sql, params or {})
    except Exception as e:
        print(f"(ignore) {e}")

def run_schema_setup():
    # Drops
    for stmt in DROP_OBJECTS_SQL:
        safe_execute(stmt)
    print("✅ Dropped any existing objects")

    # Creates
    cursor.execute(CREATE_CLIENTS)
    cursor.execute(CREATE_LOAN_APPLICATIONS)
    cursor.execute(CREATE_MOCK_LOAN_DATA)
    cursor.execute(CREATE_CLIENT_DEBT)
    cursor.execute(CREATE_CLIENTS_TO_LOAN)
    cursor.execute(CREATE_CLIENTS_TO_LOAN_RECS)
    cursor.execute(CREATE_LOAN_CHUNK)
    cursor.execute(CREATE_FUNDING_PROVIDER_TERMS)
    cursor.execute(CREATE_LENDER_TERMS)
    cursor.execute(CREATE_AFFORDABLE_HOUSING_ZONE)
    cursor.execute(CREATE_FLOODZONE)
    connection.commit()
    print("✅ Tables created")

    # Duality View
    cursor.execute(CREATE_CLIENTS_DV)
    connection.commit()
    print("✅ JSON Duality View clients_dv created")

    # ONNX model
    cursor.execute(ONNX_MODEL_SQL)
    connection.commit()
    print("✅ DEMO_MODEL loaded (or already present)")

def insert_seed_data():
    # Affordable housing
    cursor.executemany("""
        INSERT INTO AFFORDABLE_HOUSING_ZONE (LAND_TRACT_ID, ZIPCODE) VALUES (:1, :2)
    """, AFFORDABLE_HOUSING_ZONE_DATA)

    # Clients
    cursor.executemany("""
        INSERT INTO CLIENTS (CUSTOMER_ID, FIRST_NAME, LAST_NAME, CITY, STATE, ZIP_CODE, AGE, INCOME, VETERAN)
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9)
    """, CLIENTS_DATA)

    # Loan applications
    cursor.executemany("""
        INSERT INTO LOAN_APPLICATIONS (
            CUSTOMER_ID, APPLICATION_ID, REQUESTED_LOAN_AMOUNT, CREDIT_SCORE, ZIPCODE,
            LOAN_PURPOSE, LOAN_STATUS, STUDENT_STATUS, EDUCATION_LEVEL,
            FINAL_DECISION, RECOMMENDATIONS, CREDIT_RANK, TOTAL_DEBT
        ) VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13)
    """, LOAN_APPLICATIONS_DATA)

    # Mock loan data (subset shown; append the rest freely)
    cursor.executemany("""
        INSERT INTO MOCK_LOAN_DATA (
            LOAN_ID, LOAN_PROVIDER_NAME, LOAN_TYPE, INTEREST_RATE, ORIGINATION_FEE,
            TIME_TO_CLOSE, CREDIT_SCORE, DEBT_TO_INCOME_RATIO, INCOME, DOWN_PAYMENT_PERCENT, IS_FIRST_TIME_HOME_BUYER
        ) VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11)
    """, MOCK_LOAN_DATA)

    # Client debt
    cursor.executemany("""
        INSERT INTO CLIENT_DEBT (CUSTOMER_ID, APPLICATION_ID, DEBT_TYPE, DEBT_AMOUNT)
        VALUES (:1,:2,:3,:4)
    """, CLIENT_DEBT_DATA)

    # Client->Loan recs
    cursor.executemany("""
        INSERT INTO CLIENTS_TO_LOAN_RECOMMENDATIONS (CUSTOMER_ID, LOAN_ID, ACTION_NEEDED)
        VALUES (:1,:2,:3)
    """, CLIENTS_TO_LOAN_RECS_DATA)

    # Derive total_debt per application
    cursor.execute("""
        UPDATE LOAN_APPLICATIONS la
        SET total_debt = (
            SELECT NVL(SUM(cd.debt_amount),0)
            FROM CLIENT_DEBT cd
            WHERE cd.customer_id = la.customer_id
              AND cd.application_id = la.application_id
        )
    """)

    connection.commit()
    print("✅ Seed data inserted & totals updated")


# Run everything
run_schema_setup()
insert_seed_data()
print("schema initialized.")


✅ Dropped any existing objects
✅ Tables created
✅ JSON Duality View clients_dv created
✅ DEMO_MODEL loaded (or already present)
✅ Seed data inserted & totals updated
schema initialized.


## Task 3: Fetch Customer Profile

In [4]:
def fetch_customer_data(customer_id):
    cursor.execute("SELECT data FROM clients_dv WHERE JSON_VALUE(data, '$._id') = :customer_id", {'customer_id': customer_id})
    result = cursor.fetchone()
    return json.loads(result[0]) if result and isinstance(result[0], str) else result[0] if result else None

selected_customer_id = "CUST_1000"
customer_json = fetch_customer_data(selected_customer_id)

if customer_json:
    loan_app = customer_json.get("loanApplications", [{}])[0]
    print(f"Customer: {customer_json['firstName']} {customer_json['lastName']}")
    print(f"Loan Status: {loan_app['loanStatus']}")

    desired_fields = [
        ("Customer ID", selected_customer_id),
        ("Application ID", loan_app.get("applicationId", "")),
        ("First Name", customer_json.get("firstName", "")),
        ("Last Name", customer_json.get("lastName", "")),
        ("City", customer_json.get("city", "")),
        ("State", customer_json.get("state", "")),
        ("Zip code", customer_json.get("zipCode", "")),
        ("Age", customer_json.get("age", 0)),
        ("Income", customer_json.get("income", 0)),
        ("Credit score", loan_app.get("creditScore", 600)),
        ("Requested loan amount", loan_app.get("requestedLoanAmount", 0)),
        ("Total Debt", loan_app.get("totalDebt", 0)),
        ("Loan status", loan_app.get("loanStatus", "Pending Review"))
    ]
    df_customer_details = pd.DataFrame({field_name: [field_value] for field_name, field_value in desired_fields})
    display(df_customer_details)
else:
    print("No data found for customer ID:", selected_customer_id)

Customer: James Smith
Loan Status: Pending Review


,Customer ID,Application ID,First Name,Last Name,City,State,Zip code,Age,Income,Credit score,Requested loan amount,Total Debt,Loan status
0,CUST_1000,1000,James,Smith,New York City,NY,10033,52,130000,780,200000,90000,Pending Review


## Task 4: Generate Recommendation

In [5]:
# Fetch Mock Loan Data
cursor.execute("SELECT loan_id, loan_provider_name, loan_type, interest_rate, origination_fee, time_to_close, credit_score, debt_to_income_ratio, income, down_payment_percent, is_first_time_home_buyer FROM MOCK_LOAN_DATA")
df_mock_loans = pd.DataFrame(cursor.fetchall(), columns=["LOAN_ID", "LOAN_PROVIDER_NAME", "LOAN_TYPE", "INTEREST_RATE", "ORIGINATION_FEE", "TIME_TO_CLOSE", "CREDIT_SCORE", "DEBT_TO_INCOME_RATIO", "INCOME", "DOWN_PAYMENT_PERCENT", "IS_FIRST_TIME_HOME_BUYER"])

# Generate Recommendations
def generate_recommendations(customer_id, customer_json, df_mock_loans):
    loan_app = customer_json.get("loanApplications", [{}])[0]
    available_loans_text = "\n".join([f"{loan['LOAN_ID']}: {loan['LOAN_TYPE']} | {loan['INTEREST_RATE']}% interest | Credit Score: {loan['CREDIT_SCORE']} | DTI: {loan['DEBT_TO_INCOME_RATIO']}" for loan in df_mock_loans.to_dict(orient='records')])
    customer_profile_text = "\n".join([f"- {key.replace('_', ' ').title()}: {value}" for key, value in {**customer_json, **loan_app}.items() if key not in ["embedding_vector", "ai_response_vector", "chunk_vector"]])

    prompt = f"""<s>[INST] <<SYS>>You are a Loan Approver AI. Use only the provided context to evaluate the applicant’s profile and recommend loans. Format results as plain text with numbered sections (1. Comprehensive Evaluation, 2. Top 3 Loan Recommendations, 3. Recommendations Explanations, 4. Final Suggestion). Use newlines between sections.</SYS>> [/INST]
    [INST]Available Loan Options:\n{available_loans_text}\nApplicant's Full Profile:\n{customer_profile_text}\nTasks:\n1. Comprehensive Evaluation\n2. Top 3 Loan Recommendations\n3. Recommendations Explanations\n4. Final Suggestion</INST>"""

    print("Generating AI response...")
    print(" ")
    
    genai_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=oci.config.from_file(os.getenv("OCI_CONFIG_PATH", "~/.oci/config")), service_endpoint=os.getenv("ENDPOINT"))
    chat_detail = oci.generative_ai_inference.models.ChatDetails(
        compartment_id=os.getenv("COMPARTMENT_OCID"),
        chat_request=oci.generative_ai_inference.models.GenericChatRequest(messages=[oci.generative_ai_inference.models.UserMessage(content=[oci.generative_ai_inference.models.TextContent(text=prompt)])], temperature=0.0, top_p=1.00),
        serving_mode=oci.generative_ai_inference.models.OnDemandServingMode(model_id="meta.llama-3.2-90b-vision-instruct")
    )
    chat_response = genai_client.chat(chat_detail)
    recommendations = chat_response.data.chat_response.choices[0].message.content[0].text

    return recommendations

recommendations = generate_recommendations(selected_customer_id, customer_json, df_mock_loans)
print(recommendations)

Generating AI response...
 
1. Comprehensive Evaluation

Based on the provided applicant's profile, here's a comprehensive evaluation:

- The applicant, James Smith, is a 52-year-old veteran with a PhD education level and a high income of $130,000.
- He has a good credit score of 780 and a total debt of $90,000, which is relatively low compared to his income.
- He is applying for a mortgage loan with a requested amount of $200,000.
- The applicant's debt-to-income (DTI) ratio is not explicitly provided, but based on his income and total debt, it can be estimated to be around 7% ($90,000 / $130,000), which is relatively low.

2. Top 3 Loan Recommendations

Based on the comprehensive evaluation, here are the top 3 loan recommendations for James Smith:

1. Loan ID: 26 (Military Veteran loan with 0% down, military only) - 3.47% interest
2. Loan ID: 96 (Stated income loan for self-employed) - 3.6% interest
3. Loan ID: 78 (Jumbo loan for high-value properties) - 3.99% interest

3. Recommenda

## Task 5: Chunk and store the recommendations

In [6]:
# Clean any prior chunks for this customer
cursor.execute("DELETE FROM LOAN_CHUNK WHERE CUSTOMER_ID = :cust_id", {'cust_id': selected_customer_id})
connection.commit()

# Choose your chunk sizes (add more like 200, 500 if you want)
chunk_sizes = [50]  # e.g., [50, 200, 500]

# Insert chunks using VECTOR_CHUNKS. Make CHUNK_ID unique by (size  + chunk_offset).
for size in chunk_sizes:
    insert_sql = f"""
        INSERT INTO LOAN_CHUNK (CUSTOMER_ID, CHUNK_ID, CHUNK_TEXT)
        SELECT :cust_id,
               :chunk_size + vc.chunk_offset,
               vc.chunk_text
        FROM (SELECT :rec_text AS txt FROM dual) s,
             VECTOR_CHUNKS(
               dbms_vector_chain.utl_to_text(s.txt)
               BY words
               MAX {size}
               OVERLAP 0
               SPLIT BY sentence
               LANGUAGE american
               NORMALIZE all
             ) vc
    """
    cursor.execute(
        insert_sql,
        {'cust_id': selected_customer_id, 'chunk_size': size, 'rec_text': recommendations}
    )

# Fetch chunks for preview
cursor.execute("""
    SELECT CHUNK_ID, CHUNK_TEXT
      FROM LOAN_CHUNK
     WHERE CUSTOMER_ID = :cust_id
  ORDER BY CHUNK_ID
""", {'cust_id': selected_customer_id})
rows = cursor.fetchall()

# Build a compact dataframe
def _lob_to_str(v): return v.read() if isinstance(v, oracledb.LOB) else v

items = []
for cid, ctext in rows:
    txt = _lob_to_str(ctext) or ""
    items.append({
        "CHUNK_ID": cid,
        "Chars": len(txt),
        "Words": len(txt.split()),
        "Preview": (txt[:160] + "…") if len(txt) > 160 else txt
    })

df_chunks = pd.DataFrame(items).sort_values("CHUNK_ID")
connection.commit()
print(f"✅ Task 5 complete: recommendation chunked for customer {selected_customer_id} (sizes: {chunk_sizes}).")
display(df_chunks)

✅ Task 5 complete: recommendation chunked for customer CUST_1000 (sizes: [50]).


,CHUNK_ID,Chars,Words,Preview
0,51,218,33,1. Comprehensive Evaluation Based on the provi...
1,272,186,37,- He has a good credit score of 780 and a tota...
2,459,176,30,- The applicant's debt-to-income (DTI) ratio i...
3,636,147,24,is relatively low. 2. Top 3 Loan Recommendatio...
4,787,153,27,Loan ID: 26 (Military Veteran loan with 0% dow...
5,941,205,30,Loan ID: 78 (Jumbo loan for high-value propert...
6,1149,229,39,Here's an explanation for each recommendation:...
7,1379,28,6,47% with 0% down payment. 2.
8,1408,223,38,Loan ID: 96 (Stated income loan for self-emplo...
9,1631,205,36,. 3. Loan ID: 78 (Jumbo loan for high-value pr...


## Task 6: Generate Embeddings

In [7]:
# Create Embeddings for Loan Chunks ----
cursor.execute("""
    UPDATE LOAN_CHUNK
       SET CHUNK_VECTOR = dbms_vector_chain.utl_to_embedding(
           CHUNK_TEXT,
           JSON('{"provider":"database","model":"DEMO_MODEL","dimensions":384}')
       )
     WHERE CUSTOMER_ID = :cust_id
""", {'cust_id': selected_customer_id})
connection.commit()
print("✅ Task 6 complete: embedded vectors for LOAN_CHUNK rows.")

✅ Task 6 complete: embedded vectors for LOAN_CHUNK rows.


## Task 7: Implement RAG

In [18]:
question = "What 4th loan would James qualify for?"

def vectorize_question(q):
    cursor.execute("""
        SELECT dbms_vector_chain.utl_to_embedding(
            :q,
            JSON('{"provider":"database","model":"DEMO_MODEL","dimensions":384}')
        ) FROM DUAL
    """, {'q': q})
    return cursor.fetchone()[0]

print("Processing your question using AI Vector Search across chunked recommendations...")

try:
    q_vec = vectorize_question(question)

    # Retrieve top recommendation chunks (across all sizes) for this customer
    cursor.execute("""
        SELECT CHUNK_ID, CHUNK_TEXT
          FROM LOAN_CHUNK
         WHERE CUSTOMER_ID = :cust_id
           AND CHUNK_VECTOR IS NOT NULL
         ORDER BY VECTOR_DISTANCE(CHUNK_VECTOR, :qv, COSINE)
         FETCH FIRST 4 ROWS ONLY
    """, {'cust_id': selected_customer_id, 'qv': q_vec})
    retrieved = [
        (r[0], r[1].read() if isinstance(r[1], oracledb.LOB) else r[1])
        for r in cursor.fetchall()
    ]

    if not retrieved:
        # Fallback to full text as one chunk
        retrieved = [(0, recommendations)]

    # Prepare clean context for the LLM
    cleaned = [re.sub(r'[^\w\s\d.,\-\'"]', ' ', t).strip() for _, t in retrieved]
    docs_as_one_string = "\n=========\n".join(cleaned) + "\n=========\n"

    # Rebuild available loans + customer profile (same structures used earlier)
    available_loans_text = "\n".join(
        [f"{loan['LOAN_ID']}: {loan['LOAN_TYPE']} | {loan['INTEREST_RATE']}% interest | "
         f"Credit Score: {loan['CREDIT_SCORE']} | DTI: {loan['DEBT_TO_INCOME_RATIO']} | "
         f"Origination Fee: ${loan['ORIGINATION_FEE']} | Time to Close: {loan['TIME_TO_CLOSE']} days"
         for loan in df_mock_loans.to_dict(orient='records')]
    )
    loan_app = customer_json.get("loanApplications", [{}])[0]
    customer_profile_text = "\n".join(
        [f"- {k.replace('_',' ').title()}: {v}"
         for k, v in {**customer_json, **loan_app}.items()
         if k not in ["embedding_vector","ai_response_vector","chunk_vector"]]
    )

    rag_prompt = f"""\
<s>[INST] <<SYS>>
You are AI Loan Guru. Use only the provided context to answer. Do not mention sources outside of the provided context. 
    Do NOT provide warnings, disclaimers, or exceed the specified response length.
    Keep under 300 words. Be specific and actionable. Have the ability to respond in Spanish, French, Italian, German, and Portuguese if asked.
<</SYS>> [/INST]
[INST]
Question: "{question}"

# Context (top chunks from prior AI recommendations):
{docs_as_one_string}

# Available Loan Options:
{available_loans_text}

# Applicant Profile:
{customer_profile_text}

Tasks:
1) Provide a direct answer to the question.
2) Briefly justify based on profile + loan options.
[/INST]"""

    print("Generating AI response...")

    genai_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
        config=oci.config.from_file(os.getenv("OCI_CONFIG_PATH","~/.oci/config")),
        service_endpoint=os.getenv("ENDPOINT")
    )
    chat_detail = oci.generative_ai_inference.models.ChatDetails(
        compartment_id=os.getenv("COMPARTMENT_OCID"),
        chat_request=oci.generative_ai_inference.models.GenericChatRequest(
            messages=[oci.generative_ai_inference.models.UserMessage(
                content=[oci.generative_ai_inference.models.TextContent(text=rag_prompt)]
            )],
            temperature=0.0,
            top_p=0.90
        ),
        serving_mode=oci.generative_ai_inference.models.OnDemandServingMode(
            model_id="meta.llama-3.2-90b-vision-instruct"
        )
    )
    chat_response = genai_client.chat(chat_detail)
    ai_response = chat_response.data.chat_response.choices[0].message.content[0].text
    ai_response = re.sub(r'[^\w\s\d.,\-\'"]', ' ', ai_response)

    print("\n🤖 AI Loan Guru Response:")
    print(ai_response)

    # Print which chunks were retrieved (for transparency/debug)
    print("\n📑 Retrieved Chunks Used in Response:")
    for cid, text in retrieved:
        preview = text[:140].replace("\n", " ") + ("..." if len(text) > 140 else "")
        print(f"[Chunk {cid}] : {preview}")

except Exception as e:
    print(f"RAG flow error: {e}")

Processing your question using AI Vector Search across chunked recommendations...
Generating AI response...

🤖 AI Loan Guru Response:
Based on James' profile and the available loan options, the 4th loan he would qualify for is Loan ID 96  Stated income loan for self-employed with a 3.6  interest rate.

Justification  Although James is not self-employed, his high income and good credit score make him eligible for this loan. Additionally, the interest rate of 3.6  is competitive, and the loan terms align with James' requirements and profile.

📑 Retrieved Chunks Used in Response:
[Chunk 0] : 1. Comprehensive Evaluation  Based on the provided applicant's profile, here's a comprehensive evaluation:  - The applicant, James Smith, is...
